In [34]:
import pandas as pd


places_data = pd.read_csv("final-dataset/main_dataset.csv") 
reviews_data = pd.read_csv("final-dataset/review_dataset.csv")  

user_place_reviews = pd.merge(reviews_data,places_data, on='id')

# drop review_number, lat long, address, url,status, phone, types_y, price-level,review 1-5
user_place_reviews = user_place_reviews.drop(columns=['review_number', 'latitude', 'longitude', 'address', 'url', 'status', 'phone', 'types_y', 'price-level', 'review 1', 'review 2', 'review 3', 'review 4', 'review 5'])
user_place_reviews.head()

,id,types_x,review,user_id,sentiment-score,name,primary-type,rating,rating-count
0,ChIJYcGr7GSb0S0RckePBrCWikw,"hotel, lodging","It has quite small room, and the hallway is qu...",user_2573,0.506250,Singaraja Hotel (ex- POP! Hotel Hardys Singara...,hotel,4.1,2581.0
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"indonesian_restaurant, restaurant, food","Surprisingly, a really good warung that’s hidd...",user_2062,0.606250,Warung Bik Juk,indonesian_restaurant,4.5,648.0
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"tourist_attraction, restaurant, food","Only had a fleeting visit here, came by coach,...",user_2348,0.491667,Harbour Tourist Area of Buleleng,tourist_attraction,4.3,2800.0
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"coffee_shop, cafe, food, store","One word, underrated! How come place like this...",user_1448,0.672338,Abuela Coffee,coffee_shop,4.9,164.0
4,ChIJxaITmQia0S0RyrbukE8vsJU,"tourist_attraction, place_of_worship","This temple is located in Singaraja, located i...",user_4426,0.569762,Klenteng Ling Gwan Kiong,place_of_worship,4.6,142.0


In [35]:
from sklearn.preprocessing import MultiLabelBinarizer

user_place_reviews['types_x'] = user_place_reviews['types_x'].str.split(', ')

mlb = MultiLabelBinarizer()
one_hot = mlb.fit_transform(user_place_reviews['types_x'])

one_hot_df = pd.DataFrame(one_hot, columns=mlb.classes_)

user_place_reviews = pd.concat([user_place_reviews, one_hot_df], axis=1)

user_place_reviews = user_place_reviews.drop_duplicates(subset=['user_id', 'id'])

# user_place_reviews = user_place_reviews.drop(columns=['types_x'])

user_place_reviews.head()

,id,types_x,review,user_id,sentiment-score,name,primary-type,rating,rating-count,airport,...,tourist_attraction,travel_agency,turkish_restaurant,university,vegan_restaurant,vegetarian_restaurant,vietnamese_restaurant,wedding_venue,wholesaler,zoo
0,ChIJYcGr7GSb0S0RckePBrCWikw,"[hotel, lodging]","It has quite small room, and the hallway is qu...",user_2573,0.506250,Singaraja Hotel (ex- POP! Hotel Hardys Singara...,hotel,4.1,2581.0,0,...,0,0,0,0,0,0,0,0,0,0
1,ChIJZbWX6Aia0S0R0tM3h1RZ1h8,"[indonesian_restaurant, restaurant, food]","Surprisingly, a really good warung that’s hidd...",user_2062,0.606250,Warung Bik Juk,indonesian_restaurant,4.5,648.0,0,...,0,0,0,0,0,0,0,0,0,0
2,ChIJYyHbhgia0S0RzdjNXLmcf54,"[tourist_attraction, restaurant, food]","Only had a fleeting visit here, came by coach,...",user_2348,0.491667,Harbour Tourist Area of Buleleng,tourist_attraction,4.3,2800.0,0,...,1,0,0,0,0,0,0,0,0,0
3,ChIJ6zf9LJCb0S0RFv3BdLl61ZY,"[coffee_shop, cafe, food, store]","One word, underrated! How come place like this...",user_1448,0.672338,Abuela Coffee,coffee_shop,4.9,164.0,0,...,0,0,0,0,0,0,0,0,0,0
4,ChIJxaITmQia0S0RyrbukE8vsJU,"[tourist_attraction, place_of_worship]","This temple is located in Singaraja, located i...",user_4426,0.569762,Klenteng Ling Gwan Kiong,place_of_worship,4.6,142.0,0,...,1,0,0,0,0,0,0,0,0,0


In [36]:
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# List of place type columns
place_types = [
    'airport', 'american_restaurant', 'amusement_center', 'amusement_park', 'art_gallery',
    'athletic_field', 'atm', 'bakery', 'banquet_hall', 'bar', 'barbecue_restaurant',
    'barber_shop', 'beauty_salon', 'bed_and_breakfast', 'bicycle_store', 'brazilian_restaurant',
    'breakfast_restaurant', 'brunch_restaurant', 'cafe', 'campground', 'camping_cabin',
    'car_rental', 'car_wash', 'cemetery', 'child_care_agency', 'chinese_restaurant', 'church',
    'clothing_store', 'coffee_shop', 'community_center', 'consultant', 'convention_center', 'cottage',
    'cultural_center', 'dog_park', 'event_venue', 'extended_stay_hotel', 'farm', 'farmstay',
    'fast_food_restaurant', 'finance', 'fitness_center', 'florist', 'food', 'french_restaurant',
    'furniture_store', 'general_contractor', 'gift_shop', 'golf_course', 'greek_restaurant',
    'grocery_store', 'guest_house', 'gym', 'hair_care', 'hair_salon', 'hamburger_restaurant',
    'health', 'heliport', 'hiking_area', 'hindu_temple', 'historical_landmark', 'home_goods_store',
    'home_improvement_store', 'hostel', 'hotel', 'ice_cream_shop', 'indian_restaurant',
    'indonesian_restaurant', 'italian_restaurant', 'japanese_restaurant', 'jewelry_store',
    'korean_restaurant', 'lebanese_restaurant', 'library', 'liquor_store', 'lodging', 'market',
    'meal_delivery', 'meal_takeaway', 'mediterranean_restaurant', 'mexican_restaurant',
    'middle_eastern_restaurant', 'mosque', 'motel', 'movie_theater', 'museum', 'national_park',
    'night_club', 'park', 'performing_arts_theater', 'pizza_restaurant',
    'place_of_worship', 'playground', 'preschool', 'private_guest_room', 'ramen_restaurant',
    'real_estate_agency', 'resort_hotel', 'rest_stop', 'restaurant', 'sandwich_shop', 'school',
    'seafood_restaurant', 'shopping_mall', 'spa', 'spanish_restaurant', 'sporting_goods_store',
    'sports_club', 'sports_complex', 'steak_house', 'store', 'supermarket', 'sushi_restaurant',
    'swimming_pool', 'thai_restaurant', 'tourist_attraction', 'travel_agency', 'turkish_restaurant',
    'university', 'vegan_restaurant', 'vegetarian_restaurant', 'vietnamese_restaurant',
    'wedding_venue', 'wholesaler', 'zoo', 'parking'
]


# Sum the place type columns for each user to get their preferences
user_preferences = user_place_reviews.groupby('user_id')[place_types].sum().reset_index()

# Normalize the preferences
user_preferences[place_types] = user_preferences[place_types].div(user_preferences[place_types].sum(axis=1), axis=0)

# Replace non zero values with 1
user_preferences[place_types] = user_preferences[place_types].applymap(lambda x: 1 if x > 0 else 0)

user_preferences.head()


/tmp/ipykernel_48043/2597246789.py:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  user_preferences[place_types] = user_preferences[place_types].applymap(lambda x: 1 if x > 0 else 0)


,user_id,airport,american_restaurant,amusement_center,amusement_park,art_gallery,athletic_field,atm,bakery,banquet_hall,...,travel_agency,turkish_restaurant,university,vegan_restaurant,vegetarian_restaurant,vietnamese_restaurant,wedding_venue,wholesaler,zoo,parking
0,user_1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,user_10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,user_100,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,user_1000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,user_1001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

# Hyperparameters
embedding_dim = 50

# User model
user_input = Input(shape=(len(place_types),), name='user_input')
user_embedding = Dense(embedding_dim, activation='relu')(user_input)

# Place model
place_input = Input(shape=(len(place_types),), name='place_input')
place_embedding = Dense(embedding_dim, activation='relu')(place_input)

# Concatenate user and place embeddings
merged = Concatenate()([user_embedding, place_embedding])
dense_1 = Dense(128, activation='relu')(merged)
dense_2 = Dense(64, activation='relu')(dense_1)
output = Dense(1, activation='sigmoid')(dense_2)

# Create and compile the model
model = Model(inputs=[user_input, place_input], outputs=output)
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 126)]                0         []                            
                                                                                                  
 place_input (InputLayer)    [(None, 126)]                0         []                            
                                                                                                  
 dense (Dense)               (None, 50)                   6350      ['user_input[0][0]']          
                                                                                                  
 dense_1 (Dense)             (None, 50)                   6350      ['place_input[0][0]']         
                                                                                            

In [38]:
# Ensure user_preferences and data are correctly aligned and split the data accordingly
user_preferences = user_place_reviews.groupby('user_id')[place_types].sum().reset_index()
user_preferences[place_types] = user_preferences[place_types].div(user_preferences[place_types].sum(axis=1), axis=0)

# Merge data with user preferences to align place features with user preferences
merged_data = user_place_reviews.merge(user_preferences, on='user_id', suffixes=('', '_user'))

# Prepare user features and place features for each review
user_features = merged_data[[f'{ptype}_user' for ptype in place_types]].values
place_features = merged_data[place_types].values

# Generate labels (assuming binary relevance for simplicity)
# You need to ensure your labels are the same length as user and place features
labels = np.random.randint(2, size=(len(user_features),))

# Split into train and test sets ensuring the lengths match
user_train, user_test, place_train, place_test, y_train, y_test = train_test_split(
    user_features, place_features, labels, test_size=0.3, random_state=42
)


In [39]:
# Fit the model
history = model.fit(
    [user_train, place_train],
    y_train,
    epochs=15,
    batch_size=32,
    validation_data=([user_test, place_test], y_test)
)


Epoch 1/15
676/676 [==============================] - 10s 12ms/step - loss: 0.6934 - accuracy: 0.5006 - val_loss: 0.6935 - val_accuracy: 0.4955
Epoch 2/15
676/676 [==============================] - 8s 12ms/step - loss: 0.6931 - accuracy: 0.5084 - val_loss: 0.6934 - val_accuracy: 0.4963
Epoch 3/15
676/676 [==============================] - 8s 12ms/step - loss: 0.6930 - accuracy: 0.5050 - val_loss: 0.6935 - val_accuracy: 0.4931
Epoch 4/15
676/676 [==============================] - 9s 13ms/step - loss: 0.6929 - accuracy: 0.5068 - val_loss: 0.6934 - val_accuracy: 0.5004
Epoch 5/15
676/676 [==============================] - 9s 13ms/step - loss: 0.6928 - accuracy: 0.5117 - val_loss: 0.6934 - val_accuracy: 0.5038
Epoch 6/15
676/676 [==============================] - 8s 12ms/step - loss: 0.6927 - accuracy: 0.5095 - val_loss: 0.6935 - val_accuracy: 0.4986
Epoch 7/15
676/676 [==============================] - 8s 12ms/step - loss: 0.6927 - accuracy: 0.5118 - val_loss: 0.6934 - val_accuracy: 0.497

In [49]:
model.save('user_pref_modelV2.keras')

In [47]:
def recommend_places(user_id=None, top_n=10, selected_types=None):
    # Get user preferences
    if user_id:
        user_pref = user_preferences[user_preferences['user_id'] == user_id][place_types].values
    else:
        # create a temp user_pref of all 0s
        user_pref = np.zeros(len(place_types)).reshape(1, -1)

    if selected_types:
        filtered_places = user_place_reviews[user_place_reviews[selected_types].any(axis=1)]
    else:
        filtered_places = user_place_reviews
    
    place_features = filtered_places[place_types].drop_duplicates().values
    place_ids = filtered_places['id'].drop_duplicates().values

    predictions = model.predict([np.repeat(user_pref, len(place_features), axis=0), place_features])
    
    top_indices = np.argsort(predictions[:, 0])[-top_n:][::-1]

    recommended_place_ids = place_ids[top_indices]
    
    unique_recommendations = set(recommended_place_ids)
    
  
    recommended_places = filtered_places[filtered_places['id'].isin(unique_recommendations)]
    sorted_recommendations = recommended_places.sort_values(by='rating', ascending=False)
    sorted_recommendations = sorted_recommendations.drop_duplicates(subset=['name'])
    
    return sorted_recommendations[['name', 'primary-type', 'rating']]


selected_types = ['amusement_park', 'lodging']  # List the types you want to filter by
recommendations = recommend_places(top_n=10)

print("Top Recommendations for User:")
print("---------------------------------")
for index, row in recommendations.iterrows():
    print(f"Name: {row['name']}")
    print(f"Type: {row['primary-type']}")
    print(f"Rating: {row['rating']}")
    print("---------------------------------")


22/22 [==============================] - 0s 4ms/step
Top Recommendations for User:
---------------------------------
Name: Warung Heaven Munduk
Type: restaurant
Rating: 4.9
---------------------------------
Name: Warung Resto Kintamani
Type: indonesian_restaurant
Rating: 4.9
---------------------------------
Name: Monks Kitchen Bali
Type: restaurant
Rating: 4.8
---------------------------------
Name: Good Stuff Cafe & Eco Store Amed
Type: cafe
Rating: 4.7
---------------------------------
Name: TopVibes Pool & Eatery
Type: restaurant
Rating: 4.7
---------------------------------
Name: kladi villa
Type: hotel
Rating: 4.7
---------------------------------
Name: D’Uma Amed Homestay
Type: hotel
Rating: 4.7
---------------------------------
Name: Strawberry Stop
Type: restaurant
Rating: 4.4
---------------------------------
Name: Warung Abang Sari
Type: restaurant
Rating: 4.3
---------------------------------
Name: Bali Jegeg Munduk, Hotel, Bar and Restaurant
Type: hotel
Rating: 4.0
-------